# Modelo de Identificação de Fraudes

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import datetime as dt

from scipy.stats import kstest, normaltest, anderson

from sklearn.preprocessing import PowerTransformer, MinMaxScaler

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree, metrics
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

from sklearn.model_selection import train_test_split, GridSearchCV
from imblearn.under_sampling import RandomUnderSampler, ClusterCentroids, NearMiss 
from imblearn.over_sampling import RandomOverSampler, SMOTE, ADASYN
from imblearn.combine import SMOTEENN

from sklearn.metrics import (
    confusion_matrix, 
    accuracy_score, 
    precision_score, 
    recall_score,
    precision_recall_curve,
)

import warnings
from src.config import DADOS_MASTERCARD_TRATADO

warnings.filterwarnings('ignore')
sns.set_theme(palette="bright")

In [3]:
# Importando a base

transacoes = pd.read_parquet(DADOS_MASTERCARD_TRATADO)

In [4]:
# Verificar se todos os valores podem ser convertidos para inteiro (sem valores decimais)

if (transacoes['Time'] == transacoes['Time'].astype(int)).all():
    
    # Se todos os valores são inteiros, realizar a conversão
    
    transacoes['Time'] = transacoes['Time'].astype(int)
   
else:
    print("Nem todos os valores na coluna 'Time' podem ser convertidos para inteiro.")

### Vamos ajustar a escala de duas colunas para testar o resultado
- As colunas Time e Amount têm valores max muito alto. Destoa do restante do dataset.

In [6]:
# Primeiro para a coluna Time

transacoes.Time = transacoes.Time / transacoes.Time.max()

# E então para a coluna Amount

transacoes.Amount = transacoes.Amount / transacoes.Amount.max()

In [7]:
# Separando X e y

X = transacoes.drop('Class',axis=1)
y = transacoes.Class

**Separando em treino e teste**

In [9]:
# Separando em treino e teste

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=0,stratify=y)

### Realizando o random undersampling

In [11]:
# Definindo o RandomUnderSampler

rus = RandomUnderSampler(random_state=42)

In [12]:
# Definindo a nova amostra

X_resRU, y_resRU = rus.fit_resample(X_train, y_train)

### Nos passos anteriores, tivemos como os melhores hiperparâmetros:
- **<font color='blue'>Regressão Logística:**
    - {'C': 0.001, 'solver': 'liblinear'}\
- **<font color='blue'>SVC:**
    - {'C': 100, 'gamma': 'auto', 'kernel': 'rbf'}
- **<font color='blue'>Random Forest:**
    - {'criterion': 'gini', 'max_depth': 9, 'n_estimators': 30}

In [14]:
# Regressão Logística

clf_RL = LogisticRegression(random_state=0,
                             C=0.001,
                             solver='liblinear').fit(X_resRU, y_resRU)

y_pred_RL = clf_RL.predict(X_test)

confusion_matrix(y_test,y_pred_RL)

array([[85217,  8608],
       [   10,   152]], dtype=int64)

In [15]:
# SVC

clf_SVC = SVC(random_state=0,
              probability=True,
             C=100,gamma='auto',
             kernel='rbf').fit(X_resRU, y_resRU)

y_pred_SVC = clf_SVC.predict(X_test)

confusion_matrix(y_test,y_pred_SVC)

array([[85791,  8034],
       [   12,   150]], dtype=int64)

In [16]:
recall_score(y_test,y_pred_SVC)

0.9259259259259259

In [17]:
# Random Forest

clf_RF = RandomForestClassifier(max_depth=9, 
                                random_state=0,
                                criterion='gini',
                                n_estimators=30).fit(X_resRU, y_resRU)

y_pred_RF = clf_RF.predict(X_test)

confusion_matrix(y_test,y_pred_RF)

array([[91476,  2349],
       [   19,   143]], dtype=int64)

In [18]:
# Avaliando o modelo com o recall

recall_score(y_test,y_pred_RF)

0.8827160493827161

### Podemos começar adicionando novos parâmetros

**E novamente utilizar o GridSearchCV**

In [21]:
# Adicionando novos parâmetros

parametros = {
    'C': [0.001,0.01,0.1,1,10,100,1000],
    'solver': ['newton-cg','lbfgs','liblinear','sag','saga']
}

In [22]:
# Selecionando a Regressão Logística

LogReg = LogisticRegression(random_state=42)

In [23]:
# Criando o classificador

clf_RL2 = GridSearchCV(LogReg, parametros,
                       scoring='recall'
                      ).fit(X_resRU,y_resRU)

In [24]:
# E avaliando o modelo

y_pred_RL2 = clf_RL2.predict(X_test)

confusion_matrix(y_test,y_pred_RL2)

array([[85217,  8608],
       [   10,   152]], dtype=int64)

In [25]:
# Recall apenas com C e solver

recall_score(y_test,y_pred_RL)

0.9382716049382716

In [26]:
# Recall com todos os parâmetros

recall_score(y_test,y_pred_RL2)

0.9382716049382716

In [27]:
# Fazendo o resample

cc = ClusterCentroids(random_state=42)

X_resCC, y_resCC = cc.fit_resample(X_train, y_train)

##### É possível adicionar novas métricas no scoring (como, por exemplo, adicionar a precisão além do recall)

In [29]:
# Criando o classificador

clf_RL3 = GridSearchCV(LogReg, parametros,
                       scoring=['recall','precision'],
                       refit='recall'
                      )

In [30]:
clf_RL3 = clf_RL3.fit(X_resRU,y_resRU)

In [31]:
# Avaliando o novo modelo

y_pred_RL3 = clf_RL3.predict(X_test)

confusion_matrix(y_test,y_pred_RL3)

array([[85217,  8608],
       [   10,   152]], dtype=int64)